# **Lab 4 - Random fields and the Kosambi-Karhunen-Loève expansion**

## Synthetic overview from a theoretical perspective

Let $\Omega\subset\mathbb{R}^{d}$. A random field over $\Omega$ is a family of random variables $\{Z_{x}\}_{x\in\Omega}$ indexed over $\Omega$. Equivalently, it can be thought as a random function $Z$ defined over the spatial domain $\Omega$ (rather than a collection of scalar random variables).

Specifically, let us denote by $\omega$ an arbitrary event in the probability space, so that $Z_x(\omega)$ is the value attained by $Z_x$. Then,

$$Z(x,\omega):=Z_x(\omega).$$

In particular, we see that, for each event $\omega$, the object $Z(\cdot,\omega)$ is a map $\Omega\to\mathbb{R}$.
In this sense, we can think of $Z$ as a *function-valued* random variable.


We have the following result due to D. Kosambi, K. Karhunen and Michel Loève. </br></br>

________________

**Theorem (KKL expansion).**

Assume that $\int_{\Omega}\mathbb{E}|Z(x,\cdot)|^2dx<+\infty$. Then, there exists a deterministic mean function, $\mu\in L^{2}(\Omega)$, a family of orthonormal random variables $\{\psi_{i}\}_{i=1}^{+\infty}\subset L^{2}(\Omega)$, a nonincreasing vanishing sequence $\lambda_1\ge\lambda_2\ge\dots\ge0$, and a family of scalar random variables $\{\xi_i\}_{i=1}^{+\infty}$ with

$$\mathbb{E}[\xi_i,\xi_j]=\delta_{i,j}\quad\text{and}\quad\mathbb{E}[\xi_i]=0\quad\forall i,j$$

such that

$$Z(x,\omega)=\mu(x)+\sum_{i=1}^{+\infty}\xi_i(\omega)\sqrt{\lambda_i}\psi_i(x)$$

holds $\mathbb{P}$-almost surely in $\omega$ and almost everywhere in $x$. Furthermore, for every $n\in\mathbb{N}$, the truncated field
$Z_n(x)=\mu(x)+\sum_{i=1}^{n}\xi_i\sqrt{\lambda_i}\psi_i(x)$
is the best approximation of $Z$ in terms a random field based off $n$ modes. Specifically,

$$\mathbb{E}\|Z-Z_n\|_{L^{2}(\Omega)}^2=\sum_{i>n}\lambda_i.$$

Finally, the scalars $\lambda_i$ and the functions $\psi_i$ are actually the eigenvalues and eigevectors of the covariance operator, respectively. That is,

$$\int_{\Omega}K(x,y)\psi_i(y)dy=\lambda_i\psi_i(x)\quad\forall x\in\Omega,$$

for all $i$, where

$$K(x,y):=\text{Cov}(Z(x), Z(y))=\mathbb{E}[(Z(x)-\mu(x))(Z(y)-\mu(y))].$$

________________

### Gaussian processes

Gaussian processes are a particular type of random fields which have the following additional property: for every finite number of points $x_1,\dots,x_q$, the vector $[Z(x_1),\dots,Z(x_q)]^\top\in\mathbb{R}^{q}$ is Gaussian. In particular, all the variables $Z_x=Z(x)$ are normally distributed.

In this case, it can be proven that the $\xi_i$'s in the KKL expansion of $Z$ are actually **independent standard gaussians** $\xi_i\sim\mathcal{N}(0,1)$.

Consequently, the distribution of a Gaussian random field is uniquely characterized by its mean function $\mu=\mu(x)$ and its covariance function $K=K(x,y)$. In particular, given $\mu$ and $K$ we can leverage the KKL expansion to draw samples from a Gaussian process of choice! Ideally, we should follow these steps:

1. Choose $\mu$ and $K$
2. Approximate the eigenvalues and eigenfunctions of $K$ by relying on suitable discretization techniques</br>
(e.g., finite elements, quadrature rules, etc.)
3. Sample the coefficients $\xi_i\sim\mathcal{N}(0,1)$
4. Put things together using the KKL expansion


**Note**. When the covariance function can be expressed as $K(x,y)=\kappa(|x-y|)$ for some $\kappa:\mathbb{R}\to\mathbb{R}$, the random field is said to be *isotropic* or *stationary*. In general, it can be proven that smoother choices of $\kappa$ result in smoother fields.

Classical choices of such $\kappa$ are

$$\kappa_{exp}(r)=\sigma^2 \exp\left(-\frac{r}{l}\right)\quad\text{and}\quad\kappa_{sq}(r)=\sigma^2 \exp\left(-\frac{r^{2}}{2l^2}\right)$$

respectively referred to as *exponential* and *squared exponential*. Here, $\sigma^2,l>0$ are suitable parameters commonly referred to as *variance of the field* and *correlation length*.



## Preliminary step: approximating KKL modes via Finite Elements

We use finite elements as a discretization tool (P1 continuous Lagrangian elements). Having introduced a suitable mesh over $\Omega$ with $N_h$ vertices, $\{x_i\}_{i=1}^{N_h}$,

- Each continuous function $u:\Omega\to\mathbb{R}$ will be represented by its vector of nodal values $\mathbf{u}=[u(x_1),\dots,u(x_{N_h})]^\top\in\mathbb{R}^{N_h}$;

- The integral operator $$\mathcal{K}:\varphi\mapsto \int_{\Omega}K(\cdot,y)\varphi(y)dy,$$ associated to the covariance kernel $K$, will be represented by a matrix $\mathbf{K}\in\mathbb{R}^{N_h\times N_h}.$

This will allow us to find the eigenvectors $\boldsymbol{\psi}_1,\dots,\boldsymbol{\psi}_{N_h}$ representing an approximation of the eigenfunctions $\psi_1,\dots,\psi_{N_h}$.

The pieces of code right below implement several things, three of which are:

- $\textsf{assembleM}$: a function that, given the mesh, returns the $L^{2}$ mass matrix $\mathbf{M}$. The latter is defined such that
$$\|u\|_{L^{2}(\Omega)}^{2}=\mathbf{u}^\top\mathbf{M}\mathbf{u}\quad\text{and}\quad\int_{\Omega}uv=\mathbf{v}^\top\mathbf{M}\mathbf{u},$$
whenever $u$ and $v$ are functions in the finite element space.

- $\textsf{assembleK}$: this one, given a mesh and a kernel function $\kappa$, returns a representation of the covariance kernel operator $\mathbf{K}\in\mathbb{R}^{N_h\times N_h}$

- $\textsf{KKLdecomposition}$: given a mesh and a kernel $\kappa$, returns the eigenvalues and eigenvectors of the covariance kernel operator. NB: Eigenvectors are constructed to be $\mathbf{M}$-orthonormal; eigenvalues (and eigenvectors) are sorted so that $\lambda_{i}\ge\lambda_{i+1}$. To ease further inspections, this function also returns $\mathbf{M}$ as an additional output.

### Quick intro to Finite Element representations

In [4]:
try:
     import dlroms.fespaces as fe
except:
     !pip install --no-deps git+https://github.com/NicolaRFranco/dlroms.git
     import dlroms.fespaces as fe

import numpy as np
import matplotlib.pyplot as plt
from dlroms import clc

In [5]:
# Finite elements (1d example)
mesh = fe.unitintervalmesh(10)
fe.plot(mesh)
plt.title("Mesh in 1d")
None

ModuleNotFoundError: No module named 'fenics'

In [ ]:
Vh = fe.space(mesh, 'CG', 1) # Finite element space associated to the mesh
clc()

fe.dofs(Vh)

In [ ]:
u = np.array([0, 1, 2, 2, 0, -1, -1, 1, 1, 1, 0])
plt.figure(figsize = (4, 3))
fe.plot(u, Vh, axis = "on")

In [ ]:
# Finite elements (2d example)
mesh = fe.unitsquaremesh(2, 2)
plt.figure(figsize = (4, 3))
fe.plot(mesh)
plt.title("Mesh in 2d")
None

In [ ]:
Vh = fe.space(mesh, 'CG', 1)
clc()

fe.dofs(Vh)

In [ ]:
u = np.array([0, 0, 0, 0, 0, 1, 1, 1, 2])
plt.figure(figsize = (4, 4))
fe.plot(u, Vh, axis = "on")

## KKL implementation

<mark>**Exercise 1**</mark></br>

Complete the definition of the functions $\textsf{assembleK}$ and $\textsf{KKLdecomposition}$ by filling the gaps.

In [ ]:
def assembleM(Vh):
  from fenics import assemble, dx, TrialFunction, TestFunction
  u, v = TrialFunction(Vh), TestFunction(Vh) # symbols
  M = assemble(u*v*dx).array()               # bilinear form is assembled and converted to array
  clc()
  return M

def assembleK(k, Vh):
  import inspect
  if(len(inspect.signature(k).parameters)!=1):
    raise RuntimeError("This function accepts isotropic kernels only (k should be a function of a single argument).")

  vertices = fe.dofs(Vh)             # Nh x d array listing all vertices
  dim = vertices.shape[1]              # spatial dimension

    # Constructing the matrix C with Cij = k(|x_i-x_j|)
  # TO DO...
  # TO DO...
  # TO DO...

  # Adjusting to include quadrature rules (K is an integral operator!)
  # TO DO...
  # TO DO...
  # TO DO...

  return K

In [ ]:
def KKLdecomposition(kernel, mesh):
  from scipy.linalg import eigh
  K = assembleK(kernel, mesh)
  M = assembleM(mesh)

  # Solving the eigenvalue problem (with M-orthogonality)
  # TO DO...
  # TO DO...
  # TO DO...

  nmax = sum(lambdas>0)               # negative values may appear due to floating point errors
  lambdas = lambdas[:nmax]
  psis = psis[:, :nmax]
  return lambdas, psis, M

<mark>**Exercise 2.1 (Test case with $\;d=1$)**</mark></br>

Check your code on the following 1D example where $\Omega=(0,1)$ and $\kappa(r)=e^{-r/2}.$

Aside from plotting, double-check that the approximated eigenfunctions $\psi_i$ are orthonormal in the $L^{2}$ sense.

In [ ]:
mesh = fe.unitintervalmesh(500)
Vh = fe.space(mesh, 'CG', 1)
clc()

kernel = lambda r: np.exp(-0.5*r)
lambdas, psis, M = KKLdecomposition(kernel, Vh)

In [ ]:
plt.figure(figsize = (12, 3))
for i in range(4):
  plt.subplot(1,4,i+1)
  fe.plot(psis[:, i], Vh)
  plt.title("Mode #%d" % (i+1))

plt.tight_layout() # ensures a nicer visualization when using multiple subplots

In [ ]:
# Checking L2-orthonormality
# TO DO...
# TO DO...

<mark>**Exercise 2.2 (Test case with $\;d=2$)**</mark></br>

Check your code on the following 2D example where $\Omega=(0,1)\times(0,1)$ and $\kappa(r)=e^{-10r^2}.$

Aside from plotting, double-check that the approximated eigenfunctions $\psi_i$ are orthonormal in the $L^{2}$ sense.

In [ ]:
mesh = fe.unitsquaremesh(30, 30)
Vh = fe.space(mesh, 'CG', 1)
clc()

kernel = lambda r: np.exp(-10*r**2)
lambdas, psis, M = KKLdecomposition(kernel, Vh)

In [ ]:
plt.figure(figsize = (12, 3))
for i in range(4):
  plt.subplot(1,4,i+1)
  fe.plot(psis[:, i], Vh, cmap='jet', levels = 50, colorbar = True, shrink = 0.7)
  plt.title("Mode #%d" % (i+1))

plt.tight_layout()

In [ ]:
# Checking L2-orthonormality
# TO DO...
# TO DO...

## Sampling from a Gaussian process

Now, we have all the tools for sampling. In fact, given a mesh, a kernel function $\kappa$ and a mean function $\mu$, we can easily obtain a KKL expansion. Then, we simply need to sample independently the coefficients $\xi_i\sim\mathcal{N}(0,1)$ and compute the sum

$$\mathbf{z}=\boldsymbol{\mu}+\sum_{i=1}^{N_h}\xi_i\sqrt{\lambda_i}\boldsymbol{\psi}_i.$$

For simplicity, hereon, we shall assume $\boldsymbol{\mu}=0$ (centered Gaussian process).

<mark>**Exercise 3**</mark></br>

Write a function called $\textsf{KKLcombine}$ that, given

- a vector of KKL eigenvalues $\boldsymbol{\lambda}=[\lambda_1,\dots,\lambda_n]$

- a matrix of KKL eigenvectors $\boldsymbol{\Psi}=[\boldsymbol{\psi}_1,\dots,\boldsymbol{\psi}_n]\in\mathbb{R}^{N_h\times n}$,

- a matrix of KKL random coefficients $\mathbf{\Xi}=(\xi_{i,j})\in\mathbb{R}^{n\times N}$, where $\xi_{i,j}$ is the $i$th random coefficient for the $j$th simulation,

returns a sample matrix $\mathbf{Z}\in\mathbb{R}^{N_h\times N}$ where each column is a random realization of the Gaussian process with associated KKL expansion.

Test your implementation on the two test cases right below $(d=1,2)$. Try playing with the kernel function (different $\kappa$ or different correlation lenghts) to see how the latter affects the smothness and the variability of the trajectories.

In [ ]:
def KKLcombine(xis, lambdas_kkl, psis_kkl):
  # TO DO...
  # TO DO...
  # TO DO...
  return

Case $d=1$

In [ ]:
mesh = fe.unitintervalmesh(500)
Vh = fe.space(mesh, 'CG', 1)
clc()

kernel = lambda r: np.exp(-10*r**2)
lambdas, psis, M = KKLdecomposition(kernel, Vh)

In [ ]:
N = 5
np.random.seed(0)
xis = # TO DO...
Z = KKLcombine(xis, lambdas, psis)

plt.figure(figsize = (5, 3))
plt.title("Randomly sampled trajectories")

for i in range(N):
  fe.plot(Z[:, i], Vh)

seg = lambda a, b, t: (1-t)*a + t*b
zmin, zmax = Z.min(), Z.max()
plt.axis([0, 1, seg(zmin, zmax, -0.1), seg(zmin, zmax, 1.1)])
plt.axis("on")
None

Case $d>1$

In [ ]:
mesh = fe.unitsquaremesh(30, 30)
Vh = fe.space(mesh, 'CG', 1)
clc()

kernel = lambda r: np.exp(-10*r**2)
lambdas, psis, M = KKLdecomposition(kernel, Vh)

In [ ]:
N = 4
np.random.seed(1)
xis = # TO DO...
Z = KKLcombine(xis, lambdas, psis)

In [ ]:
plt.figure(figsize = (12, 3))
for i in range(4):
  plt.subplot(1,4,i+1)
  fe.plot(Z[:, i], Vh, levels = 50, colorbar = True, shrink = 0.7)
  plt.title("Sample #%d" % (i+1))

plt.tight_layout()

## Double-checking the theory: KKL truncation and field energy

As we mentioned the KKL expansion enjoys a useful optimality property related to its truncation. Specifically, for every $n\in\mathbb{N}$, the truncated field
$Z_n(x)=\mu(x)+\sum_{i=1}^{n}\xi_i\sqrt{\lambda_i}\psi_i(x)$
is the best approximation of $Z$ in terms a random field based off $n$ modes. Furthermore, one has the following formula

$$\mathbb{E}\|Z-Z_n\|_{L^{2}(\Omega)}^2=\sum_{i>n}\lambda_i,$$

which can be very helpful for determining a good truncation index $n$ apriori. Typically, $\sum_{i=1}^{N_h}\lambda_i=\mathbb{E}\|Z\|_{L^{2}(\Omega)}^2$ is called *energy of the field*, whereas $\sum_{i>n}\lambda_i$ is referred to as *residual energy* and corresponds to the variability in $Z$ that is not captured by $Z_n$.

<mark>**Exercise 4**</mark></br>

Consider the previous 2D example with $\Omega=(0,1)^2$ and $\kappa(r)=e^{-10r^2}$. Choose a truncation index $n$ by looking at the decay of the eigenvalues $\lambda_i$.

- What is the residual energy? What proportion of the field energy would be captured by an $n$ truncation of the KKL expansion?

- Leveraging the function $\textsf{KKLcombine}$, sample 1000 random realizations of both the field $Z$ and the truncated field $Z_n$. Make sure that the two samples are related (up to truncation, each simulation should correspond to the same $\xi_i$'s). Compare them visually, then estimate the MSE $\mathbb{E}\|Z-Z_n\|_{L^{2}(\Omega)}^2$ and verify the energy formula.

In [ ]:
ticks = np.logspace(0, np.log10(len(lambdas)), 6).astype("int")

plt.figure(figsize = (5, 3))
plt.semilogx(np.arange(len(lambdas))+1, lambdas)
ntrunk = 15
plt.semilogx(np.arange(ntrunk)+1, lambdas[:ntrunk])
plt.xticks(ticks, ticks)
plt.title("Eigenvalues decay")
plt.show()

In [ ]:
# Residual energy and captured proportion
# TO DO...
# TO DO...
# TO DO...

In [ ]:
# Sampling Z and Zn
# TO DO...
# TO DO...
# TO DO...

In [ ]:
# Compute average of the squared L2-norm of (Z-Zn) and compare with residual energy
# TO DO...
# TO DO...
# TO DO...

In [ ]:
for i in range(4):
  plt.figure(figsize = (6, 2.5))
  plt.subplot(1,2,1)
  vmin, vmax = Z[:, i].min(), Z[:, i].max()
  vmin, vmax = seg(vmin, vmax, -0.05), seg(vmin, vmax, 1.05)
  fe.plot(Z[:, i], Vh, cmap='jet', levels = 50, vmin = vmin, vmax = vmax, colorbar = True, shrink = 0.7)
  plt.title("Sample #%d (true)" % (i+1))
  plt.subplot(1,2,2)
  fe.plot(Zn[:, i], Vh, cmap='jet', levels = 50, vmin = vmin, vmax = vmax, colorbar = True, shrink = 0.7)
  plt.title("Sample #%d (truncated)" % (i+1))
  plt.tight_layout()